In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip install wfdb
import wfdb

import simple_denoise
from simple_denoise import get_valid_segments


FILEPATH = 'dat/1002'

signals, fields = wfdb.rdsamp(FILEPATH)

print('nSamples: {}'.format(signals.shape))

# transpose the matrix as originally it was vertical
signals = np.transpose(signals)

print(signals[0])

fetal_hr = signals[0]

# Calculate the number of samples corresponding to one minute
Fs = 1000  # Sampling frequency (adjust this according to your data)
duration_seconds = 60  # One minute
num_samples_one_minute = int(Fs * duration_seconds)
ts = np.arange(len(fetal_hr))/4.0

# Plot the first one minute of waveform data
plt.plot(fetal_hr[:num_samples_one_minute])
plt.xlabel('Sample Index')
plt.ylabel('Amplitude')
plt.title('Whole record of waveform data')
plt.grid(True)
plt.show()

selected_segments = get_valid_segments(fetal_hr, ts, 1002, verbose=True,
                                           #max_change=15, verbose_details=True
                                          )


FileNotFoundError: [Errno 2] No such file or directory: '/Users/angelinasisixia1/Desktop/School/CTUCHB/fetal-health-classification/CTU-CHB/ctu-chb/1002.hea'